In [1]:
from pynq import Overlay
from pynq import MMIO
from pynq.lib.iic import AxiIIC
import cffi, time
import threading
import csv
import math

overlay = Overlay("/home/xilinx/pynq/overlays/heading_error_ffnn.bit")
print(overlay.ip_dict.keys())

dict_keys(['top_heading_error_ff_0', 'axi_iic_0', 'processing_system7_0'])


In [2]:
overlay.ip_dict['top_heading_error_ff_0']


{'type': 'xilinx.com:hls:top_heading_error_ffnn:1.0',
 'mem_id': 's_axi_control',
 'memtype': 'REGISTER',
 'gpio': {},
 'interrupts': {},
 'parameters': {'C_S_AXI_CONTROL_ADDR_WIDTH': '6',
  'C_S_AXI_CONTROL_DATA_WIDTH': '32',
  'Component_Name': 'design_1_top_heading_error_ff_0_0',
  'II': '329',
  'clk_period': '10',
  'combinational': '0',
  'latency': '328',
  'machine': '64',
  'EDK_IPTYPE': 'PERIPHERAL',
  'C_S_AXI_CONTROL_BASEADDR': '0x40000000',
  'C_S_AXI_CONTROL_HIGHADDR': '0x4000FFFF',
  'ADDR_WIDTH': '6',
  'ARUSER_WIDTH': '0',
  'AWUSER_WIDTH': '0',
  'BUSER_WIDTH': '0',
  'CLK_DOMAIN': 'design_1_processing_system7_0_0_FCLK_CLK0',
  'DATA_WIDTH': '32',
  'FREQ_HZ': '100000000',
  'HAS_BRESP': '1',
  'HAS_BURST': '0',
  'HAS_CACHE': '0',
  'HAS_LOCK': '0',
  'HAS_PROT': '0',
  'HAS_QOS': '0',
  'HAS_REGION': '0',
  'HAS_RRESP': '1',
  'HAS_WSTRB': '1',
  'ID_WIDTH': '0',
  'INSERT_VIP': '0',
  'MAX_BURST_LENGTH': '1',
  'NUM_READ_OUTSTANDING': '1',
  'NUM_READ_THREADS': '1'

In [3]:
ADDR_BASE = 0x41600000
ADDR_RANGE = 0x10000
i2c_obj = MMIO(ADDR_BASE,ADDR_RANGE)
FFNN_ADDR_BASE = 0x40000000
FFNN_ADDR_RANGE = 0x10000
ffnn_obj = MMIO(FFNN_ADDR_BASE,FFNN_ADDR_RANGE)


In [4]:
i2c_obj.write(292,0)

In [5]:
iic_ip = overlay.ip_dict['axi_iic_0']
iic = AxiIIC(iic_ip)

ffi = cffi.FFI()

In [6]:
def read_(addr,reg,length=1):
    iic.send(addr,bytes([reg]),1,0)
    buf = ffi.new("unsigned char[]",length)
    iic.receive(addr, buf, length, 0)
    return [buf[i] for i in range(length)]

def write_(addr, reg, value):
    iic.send(addr, bytes([reg,value]), 2, 0)

def to_int16(low,high):
    val = (high << 8) | low
    if val & 0x8000:
        val -= 1 << 16
    return val

#하드웨어에 입력으로 넣기 위해 fixed16_8로 변환하는 함수
def float_to_fixed16_8(x):
    # 8 fractional bits → 256 곱함
    val = int(round(x * 256))  # 소수 반올림
    # 16비트 범위로 클리핑
    if val > 32767:
        val = 32767
    elif val < -32768:
        val = -32768
    return val


In [7]:
# BN0055 connection check
BN0055_addr = 0x28
chip_addr = 0x00
chip_id = read_(BN0055_addr, 0x00, 1)[0]
print("CHIP ID:", hex(chip_id)) #0xA0

CHIP ID: 0xa0


In [8]:
#NDOF mode on 0x0C

write_(BN0055_addr, 0x3D, 0X0C)
time.sleep(0.1)

In [9]:
#calibration check
cal = read_(BN0055_addr, 0x35, 1)[0]
sys = (cal >> 6) & 0x03
gyro = (cal >> 4) & 0x03
accel = (cal >> 2) & 0x03
mag = cal & 0x03

print(f"Cal : sys = {sys} Gyro = {gyro} accel = {accel} mag = {mag}")

Cal : sys = 0 Gyro = 3 accel = 3 mag = 0


In [10]:
# roboclaw encoder 
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

for port in ports:
    print(f"Device: {port.device} | Description: {port.description}")

import sys
sys.path.append("/home/xilinx/pynq/roboclaw")  # 실제 roboclaw_3.py가 있는 경로
from roboclaw_3 import Roboclaw

rc = Roboclaw("/dev/ttyACM0", 38400)  # 또는 ttyUSB0
rc.Open()

rc_address = 0x80

start_time_motor = time.time()

Device: /dev/ttyACM0 | Description: USB Roboclaw 2x30A


In [11]:
# ======================
# CSV 준비
# ======================
csv_file = "rover_data.csv"
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([
        "time","label", "delta_encL", "delta_encR",
        "heading", "cmdL", "cmdR"
    ])


In [12]:
def main_loop():
    # 전역 변수 선언: 모터 제어 및 센서 로깅에 필요한 모든 변수를 선언합니다.
    global pattern_idx, start_time_motor, cmdL, cmdR
    global prev_encL, prev_encR
    
    prev_encL, prev_encR = rc.ReadEncM1(rc_address)[1], rc.ReadEncM2(rc_address)[1]
    
    data = read_(BN0055_addr, 0x08, 44)
    #initial_heading = to_int16(data[18], data[19]) / 16.0
    initial_heading = 340
    print(initial_heading)
    
    cmdL, cmdR = 15,15
    
    rc.ForwardM1(rc_address, cmdL)
    rc.ForwardM2(rc_address, cmdR)
    
    LOOP_PERIOD = 0.05
    
    
    # 목표 주기 설정 (20Hz = 0.05초)
    LOOP_PERIOD = 0.05 
    CORRECTION_PERIOD = 0.15
    
    last_correction_time = time.time()
    
    while True:
        loop_start_time = time.time()
        

        
        curr_time = loop_start_time
        
        # 엔코더 읽기 및 delta 계산
        encL = rc.ReadEncM1(rc_address)[1]
        encR = rc.ReadEncM2(rc_address)[1]
        
        # delta 계산 (이전 값 사용)
        delta_encL = encL - prev_encL
        delta_encR = encR - prev_encR
        
        # 다음 루프를 위한 이전 값 갱신
        prev_encL, prev_encR = encL, encR

        # IMU 읽기
        data = read_(BN0055_addr,0x08,44)
        cur_heading = to_int16(data[18],data[19]) / 16.0
        
        heading_error = (initial_heading - cur_heading + 540) % 360 - 180
        
        
        input_data = [1,0,0,delta_encL/100,delta_encR/100, heading_error]   
        fixed_input = [float_to_fixed16_8(v) for v in input_data]
        print(fixed_input)
        

        # write input
        # -----------
        INPUT_ADDR = 16

        num_input_words = (len(fixed_input) + 1) // 2
        for i in range(num_input_words):
            low = fixed_input[i*2] & 0xFFFF
            if i*2 + 1 < len(fixed_input):
                high = fixed_input[i*2 + 1] & 0xFFFF
            else:
                high = 0
            word = (high << 16) | low
            ffnn_obj.write(INPUT_ADDR + i*4, word)

        # -----------------------------
        # 4. ap_start = 1
        # CTRL register offset = 0x00
        # bit0 = ap_start
        # -----------------------------
        CTRL_ADDR = 0
        AP_START = 1

        ffnn_obj.write(CTRL_ADDR, AP_START)

        # -----------------------------
        # 5. ap_done 기다리기
        # CTRL bit1 = ap_done
        # -----------------------------
        AP_DONE_MASK = 1 << 1

        while True:
            ctrl_reg = ffnn_obj.read(CTRL_ADDR)
            if ctrl_reg & AP_DONE_MASK:
                break

        # -----------------------------
        # 6. 출력 읽기
        # -----------------------------
        OUTPUT_ADDR = 32

        output_fixed = []
        num_words = 1

        for i in range(num_words):   # num_words = 필요한 word 수 (출력값 / 2)
            word = ffnn_obj.read(OUTPUT_ADDR + i)
            low  = word & 0xFFFF            # lower 16bit
            high = (word >> 16) & 0xFFFF    # upper 16bit

            # signed 처리 (16비트 signed fixed)
            if low & 0x8000:
                low = -((~low & 0xFFFF) + 1)
            if high & 0x8000:
                high = -((~high & 0xFFFF) + 1)

            output_fixed.append(low)
            output_fixed.append(high)

        def fixed_to_float16_2(x):
            return float(x) / 16384.0               # 14 fractional bits

        output_float = [fixed_to_float16_2(v) for v in output_fixed]

        print("Raw Fixed Output:", output_fixed)
        print("Converted Float Output:", output_float)
        # ---------------------------
        # cmd값 변환
        # ---------------------------
        pred_cmdL = output_float[0] * 127
        pred_cmdR = output_float[1] * 127

        print(f"predicted cmdL={pred_cmdL:.2f}, cmdR={pred_cmdR:.2f}")
        

        cmdL = round(pred_cmdL)
        cmdR = round(pred_cmdR)
            

        
        rc.ForwardM1(rc_address, cmdL)
        rc.ForwardM2(rc_address, cmdR)
        
        label = "forward"

        # CSV 기록
        with open(csv_file, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                curr_time, label, delta_encL, delta_encR, cur_heading, cmdL, cmdR
            ])
            
        # =======================# 3. 루프 시간 맞추기 (Loop Timing)# =======================
        loop_end_time = time.time()
        time_elapsed = loop_end_time - loop_start_time
        
        # 남은 시간만큼 대기하여 전체 루프 시간을 LOOP_PERIOD에 맞춥니다.
        sleep_time = LOOP_PERIOD - time_elapsed
        if sleep_time > 0:
            time.sleep(sleep_time)

In [13]:
main_loop()

340
[256, 0, 0, 0, 0, -608]
Raw Fixed Output: [1534, 909]
Converted Float Output: [0.0936279296875, 0.05548095703125]
predicted cmdL=11.89, cmdR=7.05
[256, 0, 0, -20, -31, -608]
Raw Fixed Output: [1368, 1292]
Converted Float Output: [0.08349609375, 0.078857421875]
predicted cmdL=10.60, cmdR=10.01
[256, 0, 0, -18, -13, -576]
Raw Fixed Output: [1449, 1085]
Converted Float Output: [0.08843994140625, 0.06622314453125]
predicted cmdL=11.23, cmdR=8.41
[256, 0, 0, -26, -28, -576]
Raw Fixed Output: [1449, 1292]
Converted Float Output: [0.08843994140625, 0.078857421875]
predicted cmdL=11.23, cmdR=10.01
[256, 0, 0, -31, -28, -576]
Raw Fixed Output: [1368, 1292]
Converted Float Output: [0.08349609375, 0.078857421875]
predicted cmdL=10.60, cmdR=10.01
[256, 0, 0, -31, -28, -560]
Raw Fixed Output: [1368, 1292]
Converted Float Output: [0.08349609375, 0.078857421875]
predicted cmdL=10.60, cmdR=10.01
[256, 0, 0, -33, -33, -560]
Raw Fixed Output: [1368, 1368]
Converted Float Output: [0.08349609375, 0.08

[256, 0, 0, -67, -59, -336]
Raw Fixed Output: [1919, 2028]
Converted Float Output: [0.11712646484375, 0.123779296875]
predicted cmdL=14.88, cmdR=15.72
[256, 0, 0, -69, -64, -336]
Raw Fixed Output: [1919, 2142]
Converted Float Output: [0.11712646484375, 0.1307373046875]
predicted cmdL=14.88, cmdR=16.60
[256, 0, 0, -72, -67, -336]
Raw Fixed Output: [2028, 2261]
Converted Float Output: [0.123779296875, 0.13800048828125]
predicted cmdL=15.72, cmdR=17.53
[256, 0, 0, -72, -72, -320]
Raw Fixed Output: [2028, 2386]
Converted Float Output: [0.123779296875, 0.1456298828125]
predicted cmdL=15.72, cmdR=18.49
[256, 0, 0, -72, -69, -336]
Raw Fixed Output: [2142, 2261]
Converted Float Output: [0.1307373046875, 0.13800048828125]
predicted cmdL=16.60, cmdR=17.53
[256, 0, 0, -72, -74, -352]
Raw Fixed Output: [2142, 2386]
Converted Float Output: [0.1307373046875, 0.1456298828125]
predicted cmdL=16.60, cmdR=18.49
[256, 0, 0, -77, -72, -352]
Raw Fixed Output: [2261, 2386]
Converted Float Output: [0.1380004

[256, 0, 0, -164, -159, -480]
Raw Fixed Output: [3776, 3961]
Converted Float Output: [0.23046875, 0.24176025390625]
predicted cmdL=29.27, cmdR=30.70
[256, 0, 0, -161, -159, -464]
Raw Fixed Output: [3776, 3961]
Converted Float Output: [0.23046875, 0.24176025390625]
predicted cmdL=29.27, cmdR=30.70
[256, 0, 0, -164, -156, -448]
Raw Fixed Output: [3776, 3961]
Converted Float Output: [0.23046875, 0.24176025390625]
predicted cmdL=29.27, cmdR=30.70
[256, 0, 0, -164, -154, -416]
Raw Fixed Output: [3961, 3961]
Converted Float Output: [0.24176025390625, 0.24176025390625]
predicted cmdL=30.70, cmdR=30.70
[256, 0, 0, -164, -156, -416]
Raw Fixed Output: [3961, 4152]
Converted Float Output: [0.24176025390625, 0.25341796875]
predicted cmdL=30.70, cmdR=32.18
[256, 0, 0, -166, -151, -384]
Raw Fixed Output: [4152, 3961]
Converted Float Output: [0.25341796875, 0.24176025390625]
predicted cmdL=32.18, cmdR=30.70
[256, 0, 0, -169, -156, -368]
Raw Fixed Output: [4152, 4152]
Converted Float Output: [0.253417

[256, 0, 0, -169, -172, -288]
Raw Fixed Output: [3961, 4761]
Converted Float Output: [0.24176025390625, 0.29058837890625]
predicted cmdL=30.70, cmdR=36.90
[256, 0, 0, -169, -174, -304]
Raw Fixed Output: [3961, 4761]
Converted Float Output: [0.24176025390625, 0.29058837890625]
predicted cmdL=30.70, cmdR=36.90
[256, 0, 0, -174, -179, -336]
Raw Fixed Output: [3961, 4761]
Converted Float Output: [0.24176025390625, 0.29058837890625]
predicted cmdL=30.70, cmdR=36.90
[256, 0, 0, -184, -182, -320]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -179, -184, -336]
Raw Fixed Output: [3961, 4761]
Converted Float Output: [0.24176025390625, 0.29058837890625]
predicted cmdL=30.70, cmdR=36.90
[256, 0, 0, -179, -179, -352]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -177, -179, -352]
Raw Fixed Output: [4152, 4761]
Converted Float

[256, 0, 0, -182, -187, -160]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -184, -182, -192]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -182, -179, -208]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -179, -184, -224]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -184, -182, -224]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -182, -182, -256]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -182, -184, -272]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.

[256, 0, 0, -182, -174, -320]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -177, -172, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -177, -172, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -177, -169, -352]
Raw Fixed Output: [4152, 4349]
Converted Float Output: [0.25341796875, 0.26544189453125]
predicted cmdL=32.18, cmdR=33.71
[256, 0, 0, -174, -174, -368]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -179, -174, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -177, -174, -368]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.

[256, 0, 0, -177, -172, -336]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -177, -174, -336]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -177, -174, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -179, -177, -368]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -179, -174, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -179, -184, -368]
Raw Fixed Output: [3961, 4761]
Converted Float Output: [0.24176025390625, 0.29058837890625]
predicted cmdL=30.70, cmdR=36.90
[256, 0, 0, -179, -174, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875,

[256, 0, 0, -174, -172, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -179, -174, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -177, -177, -352]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -179, -182, -336]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -179, -179, -368]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -179, -177, -368]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -179, -177, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.253417968

[256, 0, 0, -177, -166, -336]
Raw Fixed Output: [4152, 4349]
Converted Float Output: [0.25341796875, 0.26544189453125]
predicted cmdL=32.18, cmdR=33.71
[256, 0, 0, -172, -169, -336]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -174, -169, -320]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.25341796875, 0.27783203125]
predicted cmdL=32.18, cmdR=35.28
[256, 0, 0, -177, -177, -336]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -179, -179, -320]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -179, -182, -352]
Raw Fixed Output: [4152, 4761]
Converted Float Output: [0.25341796875, 0.29058837890625]
predicted cmdL=32.18, cmdR=36.90
[256, 0, 0, -179, -177, -352]
Raw Fixed Output: [4152, 4552]
Converted Float Output: [0.253417


KeyboardInterrupt



In [14]:
rc.ForwardM1(rc_address, 0)
rc.ForwardM2(rc_address, 0)

True